In [0]:
"""
Descripcion:
    Lee la información directamente de OCI, realiza las transformaciones necesarias y carga la información directo en las tablas de OCI
Subetapa: 
    483 - Interpretación de Contenido
Trámite:
    348 - Devolución de Saldos Excedentes por T.A.A.G.
Tablas input:
    CIERREN_ETL.TTSISGRAL_ETL_LEE_ARCHIVO
Tablas output:
    CIERREN_ETL.TTAFOTRAS_ETL_DEVO_SALDO_EXC
Tablas Delta:
    DELTA_INCO_01_{params.sr_folio}
    DELTA_INCO_02_{params.sr_folio}
    
Archivos SQL:
    INCO_DEX_43_BIS_EXT_0100_OCI_LEE_ARCHIVO
    INCO_DEX_43_BIS_TRN_0200_DBK_TRANSF_INFONAVIT
    DELETE
"""

In [0]:
%run "./startup"

In [0]:
params = WidgetParams({
    "sr_proceso": str,
    "sr_subproceso": str,
    "sr_subetapa": str,
    "sr_folio": str,
    "sr_tipo_archivo": str,
    "sr_id_archivo": str,
    "sr_mask_rec_trp": str,
    "sr_instancia_proceso": str,
    "sr_usuario": str,
    "sr_etapa": str,
    "sr_id_snapshot": str,
    
})
# Validar widgets
params.validate()

In [0]:
conf = ConfManager()

#Archivos SQL
query = QueryManager()

#Conexion a base de datos
db = DBXConnectionManager()

In [0]:
#Query Extraccion de información de OCI

statement = query.get_statement(
    "INCO_DEX_43_BIS_EXT_0100_OCI_LEE_ARCHIVO.sql",
    sr_id_archivo=params.sr_id_archivo,
)



db.write_delta(f"DELTA_INCO_01_{params.sr_folio}", db.read_data("default", statement), "overwrite")

if conf.debug:
    display(db.read_delta(f"DELTA_INCO_01_{params.sr_folio}"))

In [0]:
#Query Transformación de información

statement = query.get_statement(
    "INCO_DEX_43_BIS_TRN_0200_DBK_TRANSF_INFONAVIT.sql",
    sr_folio=params.sr_folio,
    sr_id_archivo=params.sr_id_archivo,
    sr_tipo_archivo=f'{params.sr_tipo_archivo}',
    sr_usuario=f'{params.sr_usuario}',
    SR_SUBPROCESO=f'{params.sr_subproceso}',
    DELTA_TABLA_NAME1 = f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}.DELTA_INCO_01_{params.sr_folio}",
)


db.write_delta(f"DELTA_INCO_02_{params.sr_folio}", db.sql_delta(statement), "overwrite")

if conf.debug:
    display(db.read_delta(f"DELTA_INCO_02_{params.sr_folio}"))

In [0]:
#INSERTA, no hace borrado en data stage

table_name = "CIERREN_ETL.TTAFOTRAS_ETL_DEVO_SALDO_EXC"

borrado = query.get_statement(
    "DELETE.sql",
    SR_FOLIO=params.sr_folio,
    table_name=table_name,
    hints="/*+ PARALLEL(8) */",
)

execution = db.execute_oci_dml(
    statement=borrado, async_mode=False
)

db.write_data(db.read_delta(f"DELTA_INCO_02_{params.sr_folio}"), table_name, "default", "append")
